In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr
from src.DataLoader import TableDataset
from functools import partial
from sklearn.utils.class_weight import compute_class_weight

from src.tools import *
from src.trainer import Trainer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
BATCH_SIZE = 3000
NUM_FEATURES = 50

In [3]:
dataset = TableDataset('../data/data_fe.csv')
feature_importance = pd.read_csv('../data/feature_imporstance_WS.csv')

features = list(feature_importance.iloc[:NUM_FEATURES]['feature_names'])
dataset.data = dataset.data[features + ['Machine failure']]

In [4]:
# split train data to train and validation parts
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [93]:
train_dataset, val_dataset = balance_val_split(dataset)

In [8]:
train_dataset.indices

[2543,
 9411,
 9249,
 2168,
 4597,
 674,
 4487,
 932,
 8606,
 1658,
 7178,
 8111,
 7389,
 1701,
 5349,
 4683,
 4725,
 9425,
 8696,
 9639,
 1758,
 2534,
 3097,
 1924,
 9321,
 2123,
 3800,
 6644,
 5850,
 2153,
 241,
 4280,
 9108,
 2982,
 6573,
 1163,
 8320,
 9630,
 4946,
 6903,
 4899,
 9762,
 8961,
 3837,
 590,
 6259,
 9607,
 7994,
 4855,
 7976,
 8033,
 6966,
 5563,
 9948,
 9367,
 9255,
 3743,
 6276,
 3810,
 9869,
 1782,
 8227,
 7858,
 1253,
 5792,
 4511,
 7373,
 5776,
 4094,
 9853,
 9328,
 3535,
 4338,
 2295,
 1022,
 7770,
 3425,
 7935,
 7170,
 5982,
 5738,
 359,
 1346,
 6768,
 7801,
 9715,
 7764,
 6452,
 1598,
 9840,
 9100,
 9281,
 3903,
 6567,
 8542,
 9973,
 6094,
 1398,
 7435,
 3553,
 7509,
 1062,
 5653,
 5426,
 8294,
 3259,
 7653,
 7665,
 6339,
 5466,
 1723,
 2213,
 8637,
 718,
 2323,
 3712,
 145,
 2756,
 8109,
 2216,
 2695,
 6607,
 4467,
 15,
 1353,
 4277,
 5009,
 2148,
 8353,
 207,
 8866,
 3003,
 1627,
 6954,
 8280,
 2965,
 4337,
 5036,
 2309,
 5458,
 2138,
 6832,
 5883,
 787,
 19

In [81]:
train_dl = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True
)

In [82]:
model = nn.Sequential(
    nn.Linear(NUM_FEATURES, 240),
    nn.ReLU(),
    nn.Linear(240, 500),
    nn.ReLU(),
    nn.Linear(500, 240),
    nn.ReLU(),
    nn.Linear(240, 120),
    nn.ReLU(),
    nn.Linear(120, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)

In [83]:
weights_classes = compute_class_weight('balanced',classes = np.unique(dataset.data['Machine failure']) , y = dataset.data['Machine failure'])

In [84]:
loss = Loss_class(nn.CrossEntropyLoss())
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=model,
                    device=device)
optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.9)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=20)

wandb_init_params = dict(
    name='data_fe_lr_1e-3',
    project="Internship_project"
)

In [85]:
trainer = Trainer(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params
                  )

In [86]:
trainer.train_model(learning_params)
wandb.finish()

Let's see how good I am...: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


Epoch: 1 of 20, 0.061 min


Let's see how good I am...: 100%|██████████| 1/1 [00:01<00:00,  1.15s/it]


Epoch: 2 of 20, 0.069 min


Let's see how good I am...: 100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


...

Let's see how good I am...: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]


Epoch: 19 of 20, 0.061 min


Let's see how good I am...: 100%|██████████| 1/1 [00:00<00:00,  1.00it/s]


Epoch: 20 of 20, 0.062 min


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
lr,█▇▆▆▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁
test_accuracy,█▁▁█████████████████
test_auc_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_f1_score,█▁▁█████████████████
test_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▃▃▁▁████████████████
train_auc_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_f1_score,▃▃▁▁████████████████
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
